## Merging hourly taxi, weather, and incoming passenger data for 2017

### Count of Taxi Pickups per hour

In [103]:
import numpy as np
from tqdm import tqdm
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [109]:
taxi_hourly_df = pd.read_csv('../clean_data/aggregate_taxis_all2.csv', index_col=False)

In [110]:
taxi_hourly_df.set_index('tpep_pickup_datetime', inplace=True)

In [111]:
taxi_hourly_df.index = pd.DatetimeIndex(taxi_hourly_df.index)

In [112]:
taxi_hourly_df = taxi_hourly_df.drop([pd.Timestamp('2014-11-02 01:00:00'), pd.Timestamp('2015-11-01 01:00:00'), pd.Timestamp('2017-03-12 02:00:00')])#, pd.Timestamp('2017-03-12 02:00:00')])


In [113]:
taxi_hourly_df.index = taxi_hourly_df.index.tz_localize('America/New_York')

In [114]:
taxi_hourly_df['2015']

,num_pickups,num_passengers
tpep_pickup_datetime,,
2015-01-01 00:00:00-05:00,82,149.0
2015-01-01 01:00:00-05:00,10,22.0
2015-01-01 02:00:00-05:00,14,26.0
2015-01-01 03:00:00-05:00,21,34.0
2015-01-01 04:00:00-05:00,19,43.0
2015-01-01 05:00:00-05:00,13,29.0
2015-01-01 06:00:00-05:00,11,13.0
2015-01-01 07:00:00-05:00,36,69.0
2015-01-01 08:00:00-05:00,86,145.0


In [115]:
cal = calendar()
holidays = cal.holidays(start=taxi_hourly_df.index.min(), end=taxi_hourly_df.index.max())

taxi_hourly_df['holiday'] = taxi_hourly_df.index.isin(holidays)

In [116]:
taxi_hourly_df.shape

(26005, 3)

### Checking missing hours

In [139]:
taxi_hourly_df_nans = taxi_hourly_df.reindex(df.index)

In [142]:
taxi_hourly_df_nans.head()

,num_pickups,num_passengers,holiday
2014-01-01 00:00:00-05:00,107.0,210.0,True
2014-01-01 01:00:00-05:00,34.0,45.0,False
2014-01-01 02:00:00-05:00,34.0,70.0,False
2014-01-01 03:00:00-05:00,12.0,18.0,False
2014-01-01 04:00:00-05:00,25.0,37.0,False


In [145]:
taxi_hourly_df_nans[taxi_hourly_df_nans[taxi_hourly_df.isnull().any(axis=1)]

,num_pickups,num_passengers,holiday
2014-01-20 02:00:00-05:00,NaN,NaN,NaN
2014-01-21 03:00:00-05:00,NaN,NaN,NaN
2014-01-22 03:00:00-05:00,NaN,NaN,NaN
2014-02-12 03:00:00-05:00,NaN,NaN,NaN
2014-02-17 03:00:00-05:00,NaN,NaN,NaN
2014-04-22 02:00:00-05:00,NaN,NaN,NaN
2014-04-29 02:00:00-05:00,NaN,NaN,NaN
2014-05-06 01:00:00-05:00,NaN,NaN,NaN
2014-05-21 01:00:00-05:00,NaN,NaN,NaN
2014-06-18 02:00:00-05:00,NaN,NaN,NaN


### Weather Data

In [117]:
def clean_weather(path):
    weather_df = pd.read_csv(path)
    weather_df["date"] = pd.DatetimeIndex(pd.to_datetime(weather_df.dt, unit="s")) \
                    .tz_localize('UTC').tz_convert('America/New_York')
    weather_df.drop_duplicates(['date'], keep='first', inplace = True)
    weather_df.set_index("date", inplace=True)
    
    weather_df.drop(["dt", "dt_iso", "city_id", "city_name", "lat", "lon", "weather_icon", "grnd_level", \
                 "sea_level", "clouds_all", "weather_id", "rain_1h", "rain_3h", "rain_24h", \
                 "rain_today", "snow_1h", "snow_3h", "snow_24h", "snow_today"], axis=1, inplace=True)
    return weather_df

In [118]:
weather_df = clean_weather("../clean_data/weather_NY.csv")

### Merge Taxi and Weather Data

In [119]:
taxi_weather = taxi_hourly_df.merge(right=weather_df, how='left', left_index=True, right_index=True)
taxi_weather.index = taxi_weather.index.tz_convert('America/New_York')

In [120]:
taxi_weather_filled = taxi_weather.fillna(method='ffill')

In [121]:
taxi_weather_filled.head()

,num_pickups,num_passengers,holiday,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,weather_main,weather_description
tpep_pickup_datetime,,,,,,,,,,,,
2014-01-01 00:00:00-05:00,107,210.0,True,269.27,267.59,270.93,1026.0,80.0,3.0,313.0,Clear,Sky is Clear
2014-01-01 01:00:00-05:00,34,45.0,False,268.90,267.04,270.93,1026.0,80.0,2.0,293.0,Clear,Sky is Clear
2014-01-01 02:00:00-05:00,34,70.0,False,268.48,266.48,270.37,1026.0,57.0,3.0,244.0,Clear,Sky is Clear
2014-01-01 03:00:00-05:00,12,18.0,False,268.33,265.93,270.37,1024.0,80.0,3.0,244.0,Clear,Sky is Clear
2014-01-01 04:00:00-05:00,25,37.0,False,268.10,265.37,270.37,1024.0,80.0,3.0,244.0,Clear,Sky is Clear


### Passenger seat data

In [122]:
def clean_seats(path):
    seats_per_hour = pd.read_csv(path, header=None)
#     print(seats_per_hour.head())
    seats_per_hour.columns = ['Time', 'Passengers']
    seats_per_hour["Time"] = pd.DatetimeIndex(seats_per_hour.Time)
    seats_per_hour.set_index("Time", inplace = True)

    #To get rid of errors surrounding daylight savings time. All of taxi pickups and passenger counts are NaNs for these times
    seats_per_hour = seats_per_hour.drop([pd.Timestamp('2014-03-09 02:00:00'), pd.Timestamp('2014-11-02 01:00:00'), pd.Timestamp('2015-03-08 02:00:00'), pd.Timestamp('2015-11-01 01:00:00'), pd.Timestamp('2016-03-13 02:00:00'), pd.Timestamp('2016-11-06 01:00:00'), pd.Timestamp('2017-03-12 02:00:00')])
    seats_per_hour.index = seats_per_hour.index.tz_localize('America/New_York')
    return seats_per_hour

In [123]:
seats_per_hour = clean_seats('../clean_data/passengers_per_hour_since_2014.csv')

### Merging all dataframes

In [124]:
merged_df = pd.merge(left=taxi_weather_filled, right=seats_per_hour, how='inner', left_index=True, right_index=True)

In [125]:
merged_df.shape

(26005, 13)

In [147]:
merged_df['Month'] = merged_df.index.month

In [150]:
merged_df['Hour'] = merged_df.index.hour

In [153]:
merged_df['Day'] = merged_df.index.dayofweek

In [155]:
merged_df.head()

,num_pickups,num_passengers,holiday,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,weather_main,weather_description,Passengers,Month,Hour,Day
2014-01-01 00:00:00-05:00,107,210.0,True,269.27,267.59,270.93,1026.0,80.0,3.0,313.0,Clear,Sky is Clear,108,1,0,2
2014-01-01 01:00:00-05:00,34,45.0,False,268.90,267.04,270.93,1026.0,80.0,2.0,293.0,Clear,Sky is Clear,0,1,1,2
2014-01-01 02:00:00-05:00,34,70.0,False,268.48,266.48,270.37,1026.0,57.0,3.0,244.0,Clear,Sky is Clear,0,1,2,2
2014-01-01 03:00:00-05:00,12,18.0,False,268.33,265.93,270.37,1024.0,80.0,3.0,244.0,Clear,Sky is Clear,0,1,3,2
2014-01-01 04:00:00-05:00,25,37.0,False,268.10,265.37,270.37,1024.0,80.0,3.0,244.0,Clear,Sky is Clear,0,1,4,2


In [156]:
merged_df.to_csv("../clean_data/combined_final_data.csv")

### Checking nan hours

In [130]:
df = pd.DataFrame({'P' : np.random.rand(30625)}, index=pd.date_range('2014-01-01', '2017-06-30', freq='h'))

In [133]:
df.index = df.index.tz_localize('EST')

In [136]:
check_nans = merged_df.reindex(df.index)

In [137]:
check_nans.isnull

<bound method NDFrame.isnull of                            num_pickups  num_passengers holiday     temp  \
2014-01-01 00:00:00-05:00        107.0           210.0    True  269.270   
2014-01-01 01:00:00-05:00         34.0            45.0   False  268.900   
2014-01-01 02:00:00-05:00         34.0            70.0   False  268.480   
2014-01-01 03:00:00-05:00         12.0            18.0   False  268.330   
2014-01-01 04:00:00-05:00         25.0            37.0   False  268.100   
2014-01-01 05:00:00-05:00         16.0            24.0   False  268.020   
2014-01-01 06:00:00-05:00         14.0            17.0   False  267.900   
2014-01-01 07:00:00-05:00         21.0            35.0   False  267.590   
2014-01-01 08:00:00-05:00         94.0           190.0   False  267.670   
2014-01-01 09:00:00-05:00        164.0           285.0   False  269.000   
2014-01-01 10:00:00-05:00        379.0           673.0   False  270.650   
2014-01-01 11:00:00-05:00        367.0           703.0   False  271.

In [132]:
merged_df.head()

,num_pickups,num_passengers,holiday,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,weather_main,weather_description,Passengers
2014-01-01 00:00:00-05:00,107,210.0,True,269.27,267.59,270.93,1026.0,80.0,3.0,313.0,Clear,Sky is Clear,108
2014-01-01 01:00:00-05:00,34,45.0,False,268.90,267.04,270.93,1026.0,80.0,2.0,293.0,Clear,Sky is Clear,0
2014-01-01 02:00:00-05:00,34,70.0,False,268.48,266.48,270.37,1026.0,57.0,3.0,244.0,Clear,Sky is Clear,0
2014-01-01 03:00:00-05:00,12,18.0,False,268.33,265.93,270.37,1024.0,80.0,3.0,244.0,Clear,Sky is Clear,0
2014-01-01 04:00:00-05:00,25,37.0,False,268.10,265.37,270.37,1024.0,80.0,3.0,244.0,Clear,Sky is Clear,0
